In [ ]:
#!git clone https://github.com/prchinmay/nonpersons_data.git
#!pip install boto3
%cd /content/nonpersons_data
!bash script.sh
#%cd /content/nonpersons_data/data/images/
#!find . -maxdepth 1 -type f |head -2000|xargs cp -t /content/drive/MyDrive/open_images/

/content/nonpersons_data
--2022-10-02 14:27:04--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘downloader.py’

downloader.py       100%[===================>]   4.14K  --.-KB/s    in 0s      

2022-10-02 14:27:04 (61.5 MB/s) - ‘downloader.py’ saved [4244/4244]



In [ ]:
#%cd /content
#!rm -r /content/drive/MyDrive/open_images/*
#!zip check.zip /content/check/*
#!cp /content/drive/MyDrive/open_images/checked.rar /content/checked.rar
#!unrar x checked.rar
#!rm -r bsl_imgs_9k/*
#!rm -r test_imgs/*
#!rm -r /content/bsl_imgs_25k_single/*
!rm -r /content/bsl_imgs_25k_multi/*


In [ ]:
%cd /content
#!cp /content/drive/MyDrive/surreal/train/char_parent.zip /content/
#!cp /content/drive/MyDrive/surreal/train/mask_parent.zip /content/
#!unzip char_parent.zip
#!unzip mask_parent.zip

!cp /content/drive/MyDrive/surreal/play/for_report/char_tgt.zip /content/
!cp /content/drive/MyDrive/surreal/play/for_report/mask_tgt.zip /content/
!unzip char_tgt.zip
!unzip mask_tgt.zip

In [ ]:
import numpy as np
import cv2
import glob
import pandas as pd
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random


In [ ]:
# For bsl
bsl_imgs = '/content/bsl_imgs_25k_multi'
imgs_src = '/content/images_src/'
char_src = '/content/char_src/'

mask = '/content/content/mask_parent'
char = '/content/content/char_parent'
bkg = '/content/bkg'
bbox_src = '/content/drive/MyDrive/surreal/play/for_report/bbox/bbox_src.csv'

bbox_train_yolo = '/content/bbox_bsl_25k_multi.txt'
bbox_test_yolo = '/content/bbox_test_open.txt'

des_w = 384
des_h = 224
n_tgt = 1000


In [ ]:
# For AUG
imgs_tgt = '/content/imgs_tgt/'

mask_tgt = '/content/mask_tgt/'
char_tgt = '/content/char_tgt/'
bkg = '/content/bkg/'
char_tsp = '/content/char_tsp/'

bbox_src = '/content/drive/MyDrive/surreal/play/for_report/bbox/bbox_src.csv'

#bbox_train_yolo = '/content/bbox_bsl_25k_multi.txt'
#bbox_test_yolo = '/content/bbox_test_open.txt'

des_w = 320
des_h = 240
n_tgt = 1000

In [ ]:
def mask2bbox(mask):
    a = np.where(mask != 0)
    bbox = [np.min(a[1]), np.min(a[0]),  np.max(a[1]), np.max(a[0])]
    return bbox

def bbox_tmp(xmin, ymin, xmax, ymax):
  return str(xmin) + "," + str(ymin) + "," + str(xmax) + "," + str(ymax) + "," + str(0) + " "

In [ ]:
list_bkg = glob.glob(bkg + "/*")
list_bkg.sort()

In [ ]:
#train variables
df_bbox_src = pd.read_csv(bbox_src)
df_bbox_src = df_bbox_src.sample(frac=1).reset_index(drop=True)
list_bkg_train = list_bkg[:len(df_bbox_src)]



In [ ]:
##SINGLE TRAIN AUG
list_chars = glob.glob(char_tgt + "/*")

for i in range(len(list_chars)):

  print(i)

  name_tgt = list_chars[i].rsplit("/")[-1][:-4]
  name_bkg = list_bkg_train[i].rsplit("/")[-1][:-4]

  char_tmp = cv2.imread(char_tgt + name_tgt + ".jpg")
  mask_tmp = np.load(mask_tgt + name_tgt + ".jpg.npy")

  b = cv2.imread(bkg + "/" + name_bkg + ".jpg")
  m = 1-np.round(mask_tmp)
  p = char_tmp*m
 
  alpha = m[:,:,0]*255
  char_save = np.dstack([char_tmp, alpha])
  #cv2_imshow(char_save)
  cv2.imwrite(char_tsp + name_tgt + ".png", char_save)

  #bbox
  gray = cv2.cvtColor(m.astype(np.uint8), cv2.COLOR_BGR2GRAY)
  cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  x,y,w,h = cv2.boundingRect(cnts[0])
  x1,y1,x2,y2 = x, y, x+w, y+h

  p = p[int(y1): int(y2), int(x1):int(x2),:]
  m = m[int(y1):int(y2), int(x1):int(x2)]

  if p.shape[1]<5 or p.shape[0]<10:
     continue

  #obtain sizes
  scale_rx = np.random.uniform(low=0.75, high=1.5, size=1)
  #scale_ry = np.random.uniform(low=0.5, high=2, size=1)
  m_w,m_h = int(scale_rx*(p.shape[1])), int(scale_rx*(p.shape[0]))
  b_w,b_h = des_w, des_h

  #resize
  #print([p.shape[1], p.shape[0], m_w, m_h])
  p = cv2.resize(p, (m_w,m_h), interpolation = cv2.INTER_AREA)
  m = cv2.resize(m, (m_w,m_h), interpolation = cv2.INTER_AREA)
  b = cv2.resize(b, (des_w,des_h), interpolation = cv2.INTER_AREA)

  #apply randomly
  x_off = np.random.uniform(low=10, high=des_w-10, size=1)
  y_off = np.random.uniform(low=100, high=des_h, size=1)
  mid_x, mid_y = int(x_off),  int(y_off)

  #obtain intervals 
  int_x_s = mid_x - int(np.ceil(m_w/2))
  int_x_e = mid_x + int(np.floor(m_w/2))
  int_y_s = mid_y - int(np.ceil(m_h/2))
  int_y_e = mid_y + int(np.floor(m_h/2))
    
  xmin = max(0, int_x_s)
  xmax = min(b_w, int_x_e)
  ymin = max(0, int_y_s)
  ymax = min(b_h, int_y_e)
 
  x_s = -min(0, int_x_s)
  x_e = m_w + (b_w - max(b_w, int_x_e))
  y_s = -min(0, int_y_s)
  y_e = m_h + (b_h - max(b_h, int_y_e))
    
  p = p[y_s:y_e, x_s:x_e, :]
  m = m[y_s:y_e, x_s:x_e]
    
  #merge
  b[ymin:ymax, xmin:xmax,:] = b[ymin:ymax, xmin:xmax,:]*(1-m) + p
  out = b

  ##save img 
  img_file_path = imgs_tgt + "/" + name_bkg + "_+_" + name_tgt + ".jpg"
  cv2.imwrite(img_file_path, out)

  ##save annot
  #annot = img_file_path + " " + bbox_tmp(xmin, ymin, xmax, ymax)
  #with open(bbox_train_yolo, 'a+') as fd:
  #  fd.write(annot)
  #  fd.write("\n")
  
  #rectangle_img = cv2.rectangle(out, (xmin, ymin), (xmax, ymax), (255,0,0), 2)
  #cv2_imshow(out)


In [ ]:
##SINGLE TRAIN
#for i in range(len(df_bbox_src)):
for i in range(1000):

  print(i)

  name_tgt = df_bbox_src.iloc[i]['names'].rsplit("/")[-1][:-4]
  name_bkg = list_bkg_train[i].rsplit("/")[-1][:-4]

  char_tmp = cv2.imread(char + "/" + name_tgt + ".jpg")
  mask_tmp = cv2.imread(mask + "/" + name_tgt + ".jpg")/255

  b = cv2.imread(bkg + "/" + name_bkg + ".jpg")
  m = np.round(mask_tmp)
  p = char_tmp*m

  alpha = m[:,:,0]*255
  char_save = np.dstack([char_tmp, alpha])
  #cv2_imshow(char_save)

 
  cv2.imwrite(char_src+name_tgt+".png", char_save)
  cv2.imwrite(imgs_src+name_tgt+".jpg", char_tmp)


  
  x1,y1,x2,y2 = df_bbox_src.iloc[i]['xmin'], df_bbox_src.iloc[i]['ymin'], df_bbox_src.iloc[i]['xmax'], df_bbox_src.iloc[i]['ymax']
    
  p = p[int(y1): int(y2), int(x1):int(x2),:]
  m = m[int(y1):int(y2), int(x1):int(x2)]

  if p.shape[1]<5 or p.shape[0]<10:
     continue

  #obtain sizes
  scale_rx = np.random.uniform(low=0.5, high=2.25, size=1)
  #scale_ry = np.random.uniform(low=0.5, high=2.5, size=1)
  m_w,m_h = int(scale_rx*1.2*(p.shape[1])), int(scale_rx*(p.shape[0]))
  b_w,b_h = des_w, des_h

  #resize
  #print([p.shape[1], p.shape[0], m_w, m_h])
  p = cv2.resize(p, (m_w,m_h), interpolation = cv2.INTER_AREA)
  m = cv2.resize(m, (m_w,m_h), interpolation = cv2.INTER_AREA)
  b = cv2.resize(b, (des_w,des_h), interpolation = cv2.INTER_AREA)

  #apply randomly
  x_off = np.random.uniform(low=10, high=des_w-10, size=1)
  y_off = np.random.uniform(low=100, high=des_h, size=1)
  mid_x, mid_y = int(x_off),  int(y_off)

  #obtain intervals 
  int_x_s = mid_x - int(np.ceil(m_w/2))
  int_x_e = mid_x + int(np.floor(m_w/2))
  int_y_s = mid_y - int(np.ceil(m_h/2))
  int_y_e = mid_y + int(np.floor(m_h/2))
    
  xmin = max(0, int_x_s)
  xmax = min(b_w, int_x_e)
  ymin = max(0, int_y_s)
  ymax = min(b_h, int_y_e)
 
  x_s = -min(0, int_x_s)
  x_e = m_w + (b_w - max(b_w, int_x_e))
  y_s = -min(0, int_y_s)
  y_e = m_h + (b_h - max(b_h, int_y_e))
    
  p = p[y_s:y_e, x_s:x_e, :]
  m = m[y_s:y_e, x_s:x_e]
    
  #merge
  b[ymin:ymax, xmin:xmax,:] = b[ymin:ymax, xmin:xmax,:]*(1-m) + p
  out = b

  ##save img 
  img_file_path = tgt_imgs + "/" + name_bkg + "_+_" + name_tgt + ".jpg"
  cv2.imwrite(img_file_path, out)

  ##save annot
  annot = img_file_path + " " + bbox_tmp(xmin, ymin, xmax, ymax)
  with open(bbox_train_yolo, 'a+') as fd:
    fd.write(annot)
    fd.write("\n")
  
  #rectangle_img = cv2.rectangle(out, (xmin, ymin), (xmax, ymax), (255,0,0), 2)
  #cv2_imshow(out)


In [ ]:
##MULTI TRAIN
for i in range(1,50):

  print(i)

  name_bkg = list_bkg_train[i].rsplit("/")[-1][:-4]
  b = cv2.imread(bkg + "/" + name_bkg + ".jpg")
  b = cv2.resize(b, (des_w,des_h), interpolation = cv2.INTER_AREA)

  img_file_path = bsl_imgs + "/" + str(i) + ".jpg"
  annot = img_file_path + " "

  for j in range(1,4):

    idx = i*j

    name_tgt = df_bbox_src.iloc[idx]['names'].rsplit("/")[-1][:-4]

    char_tmp = cv2.imread(char + "/" + name_tgt + ".jpg")
    mask_tmp = cv2.imread(mask + "/" + name_tgt + ".jpg")/255

  
    m = np.round(mask_tmp)
    p = char_tmp*m

    x1,y1,x2,y2 = df_bbox_src.iloc[idx]['xmin'], df_bbox_src.iloc[idx]['ymin'], df_bbox_src.iloc[idx]['xmax'], df_bbox_src.iloc[idx]['ymax']
    
    p = p[int(y1): int(y2), int(x1):int(x2),:]
    m = m[int(y1):int(y2), int(x1):int(x2)]

    if p.shape[1]<5 or p.shape[0]<10:
      continue

    #obtain sizes
    scale_rx = np.random.uniform(low=0.5, high=2.25, size=1)
    #scale_ry = np.random.uniform(low=0.6, high=2.25, size=1)
    m_w,m_h = int(scale_rx*1.2*(p.shape[1])), int(scale_rx*(p.shape[0]))
    b_w,b_h = des_w, des_h

    #resize
    p = cv2.resize(p, (m_w,m_h), interpolation = cv2.INTER_AREA)
    m = cv2.resize(m, (m_w,m_h), interpolation = cv2.INTER_AREA)

    #apply randomly
    x_off = np.random.uniform(low=0, high=des_w, size=1)
    y_off = np.random.uniform(low=100, high=des_h, size=1)
    mid_x, mid_y = int(x_off),  int(y_off)

    #obtain intervals 
    int_x_s = mid_x - int(np.ceil(m_w/2))
    int_x_e = mid_x + int(np.floor(m_w/2))
    int_y_s = mid_y - int(np.ceil(m_h/2))
    int_y_e = mid_y + int(np.floor(m_h/2))
    
    xmin = max(0, int_x_s)
    xmax = min(b_w, int_x_e)
    ymin = max(0, int_y_s)
    ymax = min(b_h, int_y_e)
 
    x_s = -min(0, int_x_s)
    x_e = m_w + (b_w - max(b_w, int_x_e))
    y_s = -min(0, int_y_s)
    y_e = m_h + (b_h - max(b_h, int_y_e))
    
    p = p[y_s:y_e, x_s:x_e, :]
    m = m[y_s:y_e, x_s:x_e]
    
    #merge
    b[ymin:ymax, xmin:xmax,:] = b[ymin:ymax, xmin:xmax,:]*(1-m) + p
    out = b

    annot = annot + bbox_tmp(xmin, ymin, xmax, ymax)
    rectangle_img = cv2.rectangle(out, (xmin, ymin), (xmax, ymax), (255,0,0), 2)

  ##save img 
  #cv2.imwrite(img_file_path, out)

  ##save annot
  #with open(bbox_train_yolo, 'a+') as fd:
  #  fd.write(annot)
  #  fd.write("\n")
  
  cv2_imshow(out)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%cd /content/
#!zip bsl_imgs_25k_single.zip bsl_imgs_25k_single/*
!zip bsl_imgs_25k_multi.zip bsl_imgs_25k_multi/*
#!zip bsl_imgs_15k.zip bsl_imgs_15k/*
#!zip test_imgs.zip test_imgs/*



In [ ]:
!cp bsl_imgs_25k_multi.zip /content/drive/MyDrive/surreal/play/1000_open/
#!cp bsl_imgs_25k_single.zip /content/drive/MyDrive/surreal/play/1000_open/
#!cp bsl_imgs_15k.zip /content/drive/MyDrive/surreal/play/1000_open/
#!cp bbox_bsl_15k.txt /content/drive/MyDrive/surreal/play/1000_open/
#!cp bbox_bsl_25k_single.txt /content/drive/MyDrive/surreal/play/1000_open/
!cp bbox_bsl_25k_multi.txt /content/drive/MyDrive/surreal/play/1000_open/

#!cp test_imgs.zip /content/drive/MyDrive/surreal/play/1000_open/
#!cp bbox_test_open.txt /content/drive/MyDrive/surreal/play/1000_open/

In [ ]:
#!cp -r /content/char_src /content/drive/MyDrive/surreal/play/for_report/
#!cp -r /content/imgs_src /content/drive/MyDrive/surreal/play/for_report/
#!cp -r /content/char_tsp /content/drive/MyDrive/surreal/play/for_report/
#!cp -r /content/imgs_tgt /content/drive/MyDrive/surreal/play/for_report/
!cp -r /content/bkg /content/drive/MyDrive/surreal/play/for_report/

In [ ]:
!zip /content/drive/MyDrive/surreal/play/for_report/imgs_tgt.zip imgs_tgt/*